# Preprocesado

In [23]:
from xml.dom import minidom

files = [
    "/content/TASS2017_T1_development.xml",
    "/content/TASS2017_T1_test.xml",
    "/content/TASS2017_T1_training.xml",
]

names = ['dev', 'test', 'train']
dict_dataset = {}
for file, name in zip(files,names):
    xmldoc = minidom.parse(file)
    tweets = xmldoc.getElementsByTagName("tweet")
    dataset = {'content':[], 'label':[], 'id':[]}

    for t in tweets:
        for node in t.childNodes:
            if node.nodeName == 'tweetid':
                idTweet = node.firstChild.data
            elif node.nodeName == 'content':
                contenido = node.firstChild.data.replace("\n", " ")
            elif node.nodeName == 'sentiment':
                if 'test' not in file:
                    for h in node.childNodes:
                        if h.nodeName == 'polarity':
                            if h.firstChild.nodeName == 'value':
                                polarizacion = h.firstChild.firstChild.data
        if 'test' not in file:
            dataset['content'].append(contenido)
            dataset['label'].append(polarizacion)
            dataset['id'].append(idTweet)
        else:
            dataset['content'].append(contenido)
            dataset['id'].append(idTweet)

    dict_dataset[name] = dataset

# Tokenizar

In [9]:
def mi_tokenizador(s):
    xx=[]
    x=s.split()
    for t in x:
       t = re.sub('@.*', "arroba", t)
       t = re.sub('#(.*)', "hashtag",t)
       t = re.sub('http.*', "http", t)
       t = re.sub('[0-9].*', "num", t)
       xx.append(t)
    return (xx)

In [24]:
import re
from nltk.tokenize import TweetTokenizer

clean_dict_dataset = {}

clean_train = []
clean_dev = []
clean_test = []

regex_at = re.compile(r'@+\w+' + '|' + '#+\w+')
regex_url = re.compile(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+')

for name in names:
  clean_dataset = {'content':[], 'label':[], 'id':[]}
  
  for tweet in dict_dataset[name]['content']:
      element = tweet
      for item in re.finditer(regex_at, element):
              element = regex_at.sub('>twit', element)
      for item in re.finditer(regex_url, element):
              element = regex_url.sub('>url', element)
      tt = TweetTokenizer(strip_handles=False, reduce_len=True, preserve_case=False).tokenize(element)
      clean_dataset['content'].append(' '.join(tt))
      #clean_dataset['content'].append(element)

  clean_dataset['label'] = dict_dataset[name]['label']
  clean_dataset['id'] = dict_dataset[name]['id']
  clean_dict_dataset[name] = clean_dataset


In [8]:
print(clean_dict_dataset['train']['content'][:3])
print(clean_dict_dataset['train']['label'][:3])

['- me caes muy bien - tienes que jugar más partidas al lol con russel y conmigo - por qué tan otako , deja de ser otako - haber si me muero', '> twit a . que puto mal escribo b . me sigo surrando help 3 . ha quedado raro el " cómetelo " ahí jajajaja', '> twit jajajaja la tuya y la d mucha gente seguro ! ! pero yo no puedo sin mi melena me muero']
['NONE', 'N', 'N']


# Vectorizar

In [54]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vect = TfidfVectorizer(tokenizer=casual_tokenize, max_df=0.8)
#vect = CountVectorizer(tokenizer=casual_tokenize)

train_vectors = vect.fit_transform(clean_dict_dataset['train']['content'])
dev_vectors = vect.transform(clean_dict_dataset['dev']['content'])

# Entrenamiento

In [73]:
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import neighbors

#sentiment_classifier = svm.SVC(C=50 ,kernel='linear')
#sentiment_classifier = svm.LinearSVC(C=100, tol=0.1, loss='hinge', max_iter=1000000000)
sentiment_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth = 3)
#sentiment_classifier = SGDClassifier()
#sentiment_classifier = neighbors.KNeighborsClassifier()
    
sentiment_classifier.fit(train_vectors, clean_dict_dataset['train']['label'])

GradientBoostingClassifier(random_state=0)

In [74]:
from sklearn.metrics import classification_report

predicciones = sentiment_classifier.predict(dev_vectors)

print(classification_report(clean_dict_dataset['dev']['label'], predicciones))

              precision    recall  f1-score   support

           N       0.58      0.76      0.66       219
         NEU       0.37      0.16      0.22        69
        NONE       0.29      0.19      0.23        62
           P       0.59      0.54      0.57       156

    accuracy                           0.54       506
   macro avg       0.45      0.42      0.42       506
weighted avg       0.52      0.54      0.52       506



# Test

In [44]:
test_vectors = vect.transform(clean_dict_dataset['test']['content'])
predicciones_test = sentiment_classifier.predict(test_vectors)
result = ''
for id, p in zip(clean_dict_dataset['test']['id'],predicciones_test):
  result += f"{id}\t{p}\n" #id + ' \t ' + p + ' \t '

In [45]:
print(result)

770567971701940224	N
770503386789711872	N
770502863017635840	P
770599972102348800	N
770599962216390656	P
770599405816864768	N
770453393714348032	N
770466143098241025	NONE
770536258682617856	P
770613199200620545	P
770612909105905665	N
770610067146235904	N
770633854667386880	P
770633713822601216	N
770443909260570624	P
770434045469659136	NEU
770463923342540800	N
770496679632076800	N
770456170691780608	N
770499448438915072	P
770510704080281600	P
770434546168897536	N
770081000747573248	NONE
770080539130916864	N
770080368636592129	N
770772755637764096	NONE
770789027142328326	N
770792054519656448	N
770773957775593472	N
770877057765105664	P
770942085960724480	N
770941295883853824	P
770940641601781760	P
770940052557950976	P
770948285288841216	NONE
770947728792817665	N
770947357173243906	NEU
770946843383631873	P
770943381652504576	N
770918556825427969	P
770918502865702912	P
770914817360195601	NONE
770914042567327746	N
770960812785692672	N
770960332818935809	N
770963910707019776	N
770963684705308

In [46]:
#open text file
text_file = open("/content/results.txt", "w")
 
#write string to file
text_file.write(result)
 
#close file
text_file.close()